# Crack identification procedure

In [1]:
%matplotlib widget
import ibvpy.api as ib
import matplotlib.pylab as plt
from matplotlib import cm
from bmcs_shear.dic_crack import\
    DICGrid, DICInpUnstructuredPoints, DICStateFields, DICCrackList, \
    DICAlignedGrid, DICGrid
import numpy as np
np.seterr(divide ='ignore', invalid='ignore');

In [2]:
from pathlib import Path
import traits.api as tr

  
class SomeClass(tr.HasTraits):

    def sumab_params(some_param='x'):
        def pretty_sumab(func):                                                                                     
            def inner(self, a, b):        
                self.count += 1
                print(str(some_param) + str(self.count) + ": " +  str(a) + " + " + str(b) + " is ", end="")                                                     
                return func(self,a,b)                                                                                    

            return inner                                                                                            
        return pretty_sumab
    count = tr.Int(0)
    
    @sumab_params('')                                                                                        
    def sumab(self,a,b):                                                                                             
        summed = a + b                                                                                          
        print(summed) 

In [3]:
sc = SomeClass()
sc.sumab(3,4)

1: 3 + 4 is 7


In [4]:
names = 'one',
isinstance(names,str)

False

In [5]:
dic_points = DICInpUnstructuredPoints(U_factor=10, dir_name='B1_TV1', t=1,
                                      n_T_max=40, T_stepping='delta_T')
dic_points.read_beam_design()
dic_grid = DICGrid(dic_inp=dic_points, d_x=5, d_y=5)

In [6]:
dic_points.time_F_T

(array([   0.   ,  430.595,  472.45 , 1034.117, 1110.567, 1753.873,
        1791.834, 2501.982, 2540.525, 3258.823, 3335.277, 3978.238,
        4016.127, 4602.7  , 4752.7  , 4902.7  , 4917.7  , 5642.7  ,
        5794.1  ]),
 array([-1.223000e-01,  1.856990e+01,  2.387070e+01,  3.447270e+01,
         4.507330e+01,  6.627710e+01,  7.157810e+01,  9.278200e+01,
         9.808260e+01,  1.139854e+02,  1.245870e+02,  1.404904e+02,
         1.457912e+02,  1.477462e+02,  1.627971e+02,  1.740975e+02,
         1.751882e+02,  1.862295e+02,  1.955931e+02]))

In [7]:
dsf = DICStateFields(dic_grid=dic_grid, R=10, omega_threshold=0.2, verbose_eval=True)
dsf.tmodel_.trait_set(E=1400, c_T=0.006, nu=0.18, epsilon_0=0.0018,
                      epsilon_f=0.002, eps_max=0.01);
dsf.eval()
dcl = DICCrackList(dsf=dsf, corridor_left=4, corridor_right=1, x_boundary=30)
dcl.identify_cracks()

T: 0
T: 1
T: 2
T: 3
T: 4
T: 5
T: 6
T: 7
T: 8
T: 9
T: 10
T: 11
T: 12
T: 13
T: 14
T: 15
T: 16
T: 17
T: 18
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0
ironing with 10.0


In [8]:
dcl.interact()

In [9]:
dcl.dsf.X_fe_KLa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

array([   2.00761149,   10.92675195, 1282.60288851,  337.98104805])

In [10]:
dcl.dsf.dic_grid.X_IJa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

array([1.0000000e+00, 1.0000000e+01, 1.2836105e+03, 3.3890780e+02])

In [11]:
dcl.dsf.X_ipl_MNa[(0,0,-1,-1),(0,0,-1,-1),(0,1,0,1)]

array([   2.00761149,   10.92675195, 1282.60288851,  337.98104805])

In [12]:
len(dcl.items)

8

In [13]:
%%capture

from matplotlib import animation, rc
from IPython.display import HTML
from traitlets import HasTraits
import matplotlib.gridspec as gridspec


fig = plt.figure(figsize=(12,3))

In [14]:
class CrackListAnimator(HasTraits):
    
    
    def init(self):
        print('init')
        n_T = dsf.dic_grid.n_T
        self.t_dic_T = np.hstack([
            np.linspace(0,1,n_T),
            np.ones((int(0.5*n_T),))
        ])
    
    def subplots(self, fig):
        gs = gridspec.GridSpec(ncols=2, nrows=1,
                               width_ratios=[3, 1],
                               #wspace=0.5,
                               hspace=0.5,
                               # height_ratios=[2, 1]
                               )
        ax_dcl = fig.add_subplot(gs[0, 0])
        ax_FU = fig.add_subplot(gs[0, 1])
#        return fig.subplots(1,1)
#        return ax_dsf#, ax_FU
        return ax_dcl, ax_FU

    def plot(self, i):
        fig.clear()
        t = self.t_dic_T[i]
        print('t', t)
        axes = self.subplots(fig)
        dcl.dsf.dic_grid.t = t
        
        ax_dcl, ax_FU = axes
#        ax_dsf = axes
#        ax_FU = axes
        
        dcl.bd.plot_sz_bd(ax_dcl)
        dcl.dsf.plot_crack_detection_field(ax_dcl, fig)
        dcl.plot_cracking_hist2(ax_dcl)
        dcl.critical_crack.plot_x_t_crc_Ka(ax_dcl, line_width=2, line_color='red', tip_color='red')
        ax_dcl.axis('equal')
        ax_dcl.axis('off');
        dcl.dsf.dic_grid.plot_load_deflection(ax_FU)
        
    def mp4_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.save("cracking_animation.gif")
    
    def html5_video(self):
        n_t = len(self.t_dic_T)
        # call the animator. blit=True means only re-draw the parts that have changed.
        anim = animation.FuncAnimation(fig, self.plot, init_func=self.init,
                                       frames=n_t, interval=300, blit=True)
        return anim.to_html5_video()

In [17]:
anim = CrackListAnimator()
anim.init()
anim.mp4_video()

init
init
t 0.0
t 0.05555555555555555
t 0.1111111111111111
t 0.16666666666666666
t 0.2222222222222222
t 0.2777777777777778
t 0.3333333333333333
t 0.38888888888888884
t 0.4444444444444444
t 0.5
t 0.5555555555555556
t 0.611111111111111
t 0.6666666666666666
t 0.7222222222222222
t 0.7777777777777777
t 0.8333333333333333
t 0.8888888888888888
t 0.9444444444444444
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0


In [16]:
anim = CrackListAnimator()
anim.init()
with open('cracking_animation.html','w') as html_video_file:
    html_video_file.write(anim.html5_video())

init
init
t 0.0
t 0.05555555555555555
t 0.1111111111111111
t 0.16666666666666666
t 0.2222222222222222
t 0.2777777777777778
t 0.3333333333333333
t 0.38888888888888884
t 0.4444444444444444
t 0.5
t 0.5555555555555556
t 0.611111111111111
t 0.6666666666666666
t 0.7222222222222222
t 0.7777777777777777
t 0.8333333333333333
t 0.8888888888888888
t 0.9444444444444444
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0
t 1.0


In [46]:
html_video_file = open('cracking_animation.html','r')
HTML(html_video_file.read())

In [14]:
A = (76/2)**2 * 3.14
F = 3600000

In [15]:
F / A

793.9728637719004